# __WSI - ćwiczenie 1.__
### __Zagadnienie przeszukiwania i podstawowe podejścia do niego__


1. Narysować funkcje f(x) i g(x).
2. Zaimplementować algorytm najszybszego spadku oraz zastosować go do znalezienia minimum
funkcji f i g.
3. Zbadać wpływ rozmiaru kroku dla różnych (losowych) punktów początkowych.

In [26]:
import numpy as np
from plotly import graph_objs as go
import math
import pandas as pd

RNG = np.random.default_rng()

#### __Definicje funkcji oraz ich gradientów__

In [27]:
def f(vect):
    assert vect.shape == (1,)
    return 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]
    
def g(vect):
    assert vect.shape == (2,)
    return 10*vect[1]**4 + 10*vect[0]**4 + 3*vect[0]**3 - 30*vect[0]**2 + 10*vect[0]

def grad_f(vect):
    assert vect.shape == (1,)
    return np.array(40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10)

def grad_g(vect):
    assert vect.shape == (2,)
    return np.array([40*vect[0]**3 + 9*vect[0]**2 - 60*vect[0] + 10,
                     40*vect[1]**3])

### 

#### __Algorytm najszybszego spadku__


In [28]:
def gradient_descent(start_point, beta, grad, stop_treshlod, num_iters):
    """
    Implementation of gradient descent for minimalisation purposes,
    usable with n-dimensional loss functions, if a suitable gradient 
    function is provided.

    Args:
        start_point: coordinates of the first point
        beta: learning rate of the algorithm = step coefficient
        grad: a function which returns a matrix of partial derivatives of the input function
        stop_treshlod: minimal value of gradient
        num_iters: maximal number of iterations (steps)

    Returns:
        Returns a tuple of an array of points generated by alogrithm
        (dependiong on the gradiant function the point format may vary),
        and a boolean value which depicts whether the function has succesfully
        performed all of its iterations or reached given stop treshold. 

    Raises:
        None
    """
    steps = np.array([start_point])
    point = start_point
    for _ in range(num_iters):
        theta = grad(point)
        # check if algorithm reached a local minimum,
        # i.e. func has flattened out
        # Could be changed for Euclidean distance, 
        # but this should perform faster 
        if abs(theta.sum()) < stop_treshlod:
            break
        # prevent overflow errors
        if abs(theta.sum()) > 100000:
            return steps, False
        # perform algorithm step
        point = point - beta * theta
        # add point to the output array
        steps = np.append(steps, [point], 0)
    return steps, True


#### __Generowanie wykresów__


Wykres dla funkcji jednej zmiennej f(x):

In [29]:
max_r = 3
X = np.linspace(-max_r, max_r, 100)
Y = np.array([f(np.array([x])) for x in X])

pt = RNG.uniform(-max_r, max_r, 1)
dsc = gradient_descent(pt, 0.005, grad_f, 0.01, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([x for x in dsc[0]
                    if abs(x[0]) < max_r])
    XS = steps[:, 0]
    YS = np.array([f(x) for x in steps])

    layout = go.Layout(width=700, height=500,
                       title='Gradient Descent of single variable function',
                       xaxis_title='x',
                       yaxis_title='f(x)',
                       plot_bgcolor='DarkSeaGreen')
    fig = go.Figure(data=[go.Scatter(x=X, y=Y, line=dict(color='DarkSlateGrey', width=3))], 
                    layout=layout)
    fig.add_trace(go.Scatter(x=XS, y=YS, mode='lines+markers', 
                            marker=dict(size=6, color=YS,
                            colorscale='Agsunset'),
                            line=dict(color='DarkSlateGrey', width=1)))
    fig.show()

Wykres dla funkcji dwóch zmiennych g(x, y):

In [30]:
max_r = 3
l = np.linspace(-max_r, max_r, 100)
X, Y = l, l
Z = np.array([[g(np.array([x, y])) for x in X] for y in Y])

pt = RNG.uniform(-max_r, max_r, 2)
dsc = gradient_descent(pt, 0.005, grad_g, 0.1, 100)
if not dsc[1]:
    print("Algorithm hasn't found the optimum, steps are out of bounds")
else:
    steps = np.array([pt for pt in dsc[0] if abs(pt[0]) < max_r and abs(pt[1]) < max_r])
    XS, YS = steps[:,0], steps[:,1]
    ZS = np.array([g(np.array([x, y])) for x, y in zip(XS, YS)])

    layout = go.Layout(width = 700, height =700,
                       title_text='Gradient Descent minimalisation of double variable function',
                       xaxis_title='x',
                       yaxis_title='y')                       
    fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z, colorscale='Emrld',
                                    opacity=0.5)], layout=layout)
    fig.update_traces(contours_z=dict(show=True, usecolormap=True))
    fig.add_scatter3d(x=XS, y=YS, z=ZS, mode='lines+markers', 
                      marker=dict(size=4, color=ZS,               
                                  colorscale='Agsunset'),
                      line=dict(color='DarkSlateGrey', width=0.7))
    fig.update_layout(scene=dict(zaxis_title="g(x, y)"))
    fig.show()

#### __Analiza wydajności funkcji najszybszego spadku w zależności od wartości współczynnika kroku__

Z przeprowadzonych obserwacji wywnioskowałem, że potrzebuję co najmniej kilkuset iteracji (uruchomień dla punktów początkowych) by zbadać generalne zachowanie algorytmu. Badania przeprowadzałem głównie na obszarze od -3 do 3, na którym 100 kroków algorytmu było zdecydowanie wystarczające żeby funkcja zatrzymała się po osiągnięciu dolnego limitu wartości gradientu, oznaczonego współczynnikiem odcięcia.


In [31]:
vals = [g(p) for p in dsc[0]]
layout = go.Layout(width=700, height=500,
                title_text='g(x,y) function value for GDA steps',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='Traversed point number',
                yaxis_title='g(x, y)')
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(mode='lines+markers', 
                         marker=dict(size=6,
                                     color='DarkSlateGrey'),
                         line=dict(color='DarkSlateGrey', width=1), 
                         x=list(range(len(vals))), y=vals))
fig.show()

Współczynnik odcięcia __stop_treshold__ ustawiłem na 0.01, co wydało mi się wystarczające żeby stwierdzić że funkcja wystarczajaco zbliżyła się do minimum lokalnego i się stamtąd nie ruszy.

Funkcja testująca współczynnik kroku __beta__ uruchamia algorytm gradientowy __num_iters__ razy dla losowych punktów w zakresie badanego obszzaru i zwraca listę punktów końcowych wywołań algorytmu.

In [32]:
def test_beta(func, func_parameter_num, grad, beta: int, max_r, num_iters: int):
    end_points = []
    for _ in range(num_iters):
        p = RNG.uniform(max_r[0], max_r[1], func_parameter_num)
        dsc = gradient_descent(p, beta, grad, 0.01, 100)
        if not dsc[1]:
            end_points.append(np.nan)
        else:
            end_points.append(func(dsc[0][-1]))
    return np.array(end_points)

Metodą inżynierską zbadałem, że optymalna wartość zmienia się w zależnosci od rozimaru powierzchni dopuszczalnej. Dlatego, oraz dlatego, że w przypadkach innych funkcji których wykresów nie znamy, korzystne będzie przebadać większy zakres współczynników __beta__. W tym celu tworzę _dataframe_ który posłuży mi do dalszej analizy. Każda kolumna owej tabeli odpowiada badanym współczynnikom __beta__, a w wierszach znajdują się informacje o wartościach zwracanych przez _GDA_ dla losowych  punktów startowych.

In [33]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
data = performance = {np.round(b, 4):(test_beta(g, 2, grad_g, b, [-10, 10], num_iters)) for b in b_arr}
df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]

In [34]:
df

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
0,-31.391679,-39.042275,-39.403475,-6.111814,-42.072364,-42.264879,-6.528925,-6.785445,-42.515898,NaN,...,NaN,NaN,NaN,NaN,-7.005932,-7.006025,NaN,NaN,NaN,NaN
1,7.660128,-3.090528,-5.922798,-42.544490,-6.937274,-6.858943,-42.627535,-6.988837,NaN,-6.976708,...,NaN,NaN,NaN,NaN,-40.761509,NaN,NaN,NaN,NaN,NaN
2,-27.006999,-5.548313,-5.500268,-41.836108,-6.465047,-42.235997,-6.906290,NaN,NaN,-6.998267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-27.807042,-41.295447,-5.043435,-42.627608,-6.438458,-6.726202,-42.339155,NaN,NaN,-42.486700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.005966
4,8.607263,-39.095706,-6.641646,-41.754529,-6.449966,NaN,-42.627583,-42.408700,-42.506294,-6.864937,...,NaN,NaN,NaN,NaN,-7.005949,NaN,NaN,NaN,NaN,-3.449416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-26.544582,-3.055592,-41.321004,-6.208188,-6.442982,-6.622588,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,8.939674,-4.165600,-6.261896,-42.530436,-42.273840,-6.638286,NaN,NaN,-42.458908,-6.867518,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,5.328271,0.800629,-5.951890,-41.828396,-42.207573,-42.236740,-6.719277,-6.788127,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,-1.069373,-38.950625,-41.119136,-42.250446,-6.440571,-42.427717,-6.734173,-42.407937,-6.867976,-42.483607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df.describe()

,0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.0010,...,0.0190,0.0191,0.0192,0.0193,0.0194,0.0195,0.0196,0.0197,0.0198,0.0199
count,100.000000,100.000000,100.000000,100.000000,100.000000,83.000000,70.000000,57.000000,48.000000,56.000000,...,3.000000,4.000000,5.000000,4.000000,5.000000,6.000000,1.000000,3.000000,5.000000,4.000000
mean,-9.784239,-19.477540,-20.800182,-23.306292,-26.464022,-25.553987,-30.126466,-23.703016,-28.386147,-25.347024,...,-15.965215,-13.415563,-12.092139,-10.012331,-17.615388,-27.268450,-39.987377,-27.024254,-11.561329,-7.378903
std,19.464103,17.912567,17.409050,17.859192,17.758616,17.844303,17.119851,17.937687,17.598779,17.955347,...,13.910474,12.819233,11.373113,2.747486,14.655395,16.692116,NaN,19.182182,10.186058,3.539208
min,-40.346426,-42.567089,-42.625113,-42.627608,-42.624289,-42.627588,-42.627671,-42.579323,-42.618826,-42.627678,...,-31.990407,-32.644412,-32.436982,-13.555369,-40.761509,-42.619302,-39.987377,-40.798460,-29.782703,-12.054259
25%,-26.805571,-39.132243,-41.083485,-41.788149,-42.081625,-42.253397,-42.355401,-42.415960,-42.472697,-42.490061,...,-20.444851,-13.415586,-7.006233,-11.202916,-23.594455,-41.184939,-39.987377,-37.978790,-7.006051,-8.268045
50%,-10.128027,-6.301128,-6.555757,-6.997150,-42.050908,-42.218993,-42.339238,-7.005229,-42.453207,-42.483492,...,-8.899294,-7.005956,-7.005977,-9.744001,-9.709094,-32.317460,-39.987377,-35.159120,-7.005966,-7.005969
75%,10.343324,-3.096532,-5.450524,-6.186376,-6.464120,-6.670777,-6.921043,-6.797480,-6.878326,-6.880503,...,-7.952619,-7.005933,-7.005944,-8.553416,-7.005949,-11.986672,-39.987377,-20.137151,-7.005963,-6.116828
max,15.519207,1.680779,-3.310997,-0.455239,-5.579789,-6.605488,-3.214103,-6.784176,-6.825563,-6.862278,...,-7.005944,-7.005927,-7.005562,-7.005954,-7.005932,-7.005973,-39.987377,-5.115182,-7.005961,-3.449416


Następnie rysuję wykres wartości funkcji dla punktów zwracanych przez _GDA_, dla odpowiednich wartosci współczynników __beta__. 

In [36]:
layout = go.Layout(width=700, height=500,
                   title_text='Algorithm outputs in function of beta value',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='beta',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(layout=layout)
for _, row in df.iterrows():
    fig.add_trace(go.Scatter(mode='markers', marker_color='DarkSlateGrey', 
                             opacity=0.5, x=df.columns, y=row, showlegend = False))
fig.show()

Na tym wykresie nie widać jednak ile iteracji dla każdej wartości __beta__ nie osiągneło minimum. Wobec tego następny wykres przedstawia stosunek ilości nieudanych iteracji do wykonanych.

In [37]:
fail_rate = df.isna().sum()/num_iters

layout = go.Layout(width=700, height=500,
                title_text='Beta fail (out of bounds) rate',
                plot_bgcolor='DarkSeaGreen',
                xaxis_title='beta',
                yaxis_title='failed/total iterations')
fig = go.Figure(data=[go.Bar(x=df.columns, y=fail_rate, marker_color='DarkSlateGrey')], layout=layout)
fig.show()

Następnie po ustaleniu maksymalnego błędu wyznaczam najlepszy parametr __beta__, taki, który daje możliwie najniższe wyniki _GDA_.

In [43]:
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0003


Na koniec rysuję wykres wartości osiaganych przez _GDA_ dla znalezionego parametru __beta__. 

In [39]:
layout = go.Layout(width=700, height=500,
                   title_text='Best beta outputs',
                   plot_bgcolor='DarkSeaGreen',
                   xaxis_title='Iteration number',
                   yaxis_title='min(g(x, y)) from GDA')
fig = go.Figure(data=[go.Bar(y=df[best_b], marker_color='DarkSlateGrey')], layout=layout)
fig.update_yaxes(autorange="reversed")
fig.show()

Z powyższego wykresu można ostatecznie wnioskować, że funkcja ta ma dwa minima lokalne. Dla znalezionego parametru beta powinniśmy wobec tego otrzymać wartość minimalną w okolicy _-42_ bądź _-7_.

In [52]:
p = RNG.uniform(-10, 10, 2)
arr = gradient_descent(p, best_b, grad_g, 0.0001, 1000)[0]
g_min = arr[-1]
print(f"argmin(g) = {g_min}")
print(f"Step amount: {len(arr)-1}")
print(f"g(argmin) = {g(g_min)}")

argmin(g) = [-1.41237061  0.20291578]
Step amount: 1001
g(argmin) = -42.61072512498919


Analogiczny proces przeprowadziłem dla funkcji jednej zmiennej f(x):

In [41]:
b_arr = np.arange(0.0001, 0.02, 0.0001)
num_iters = 100
performance = {np.round(b, 4):(test_beta(f, 1, grad_f, b, [-10, 10], num_iters)) for b in b_arr}

df = pd.DataFrame(columns=[b for b in performance.keys()])
for col in df.columns:
    df[col] = performance[col]
max_error = 0.1
tmp_df = df[df.columns[df.isna().sum()/num_iters < max_error]]
best_b = tmp_df.columns[tmp_df.sum().argmin()]
print(best_b)

0.0003


In [49]:
p = RNG.uniform(-10, 10, 1)
arr = gradient_descent(p, best_b, grad_f, 0.0001, 1000)[0]
f_min = arr[-1]
print(f"argmin(f) = {f_min}")
print(f"Step amount: {len(arr)-1}")
print(f"f(argmin) = {f(f_min)}")

argmin(f) = [1.01255743]
Step amount: 456
f(argmin) = -7.006322161186553


Ostatecznie, przy _10000_ iteracji dla funkcji _g(x, y)_ algorytm znalajduje minima: _g([-1.41, 0.05]) = -42.63_ oraz _g([1.01 0.05]) = -7.01_, a także dla funkcji f(x): _f([-1.41]) = -42.63_ i 
 _f([1.01]) = -7.01_, gdzie to pierwsze to minima globalne.

#### Wnioski


Ze względu na pewną ułomność tego algorytmu, oraz naturę badanych funkcji, algorytm ma tendencję wpadać w minima lokalne, a także wystrzeliwywać wyniki w kosmos kiedy pochodne badanych funkcji są duże oraz gwałtownie maleją w okolicy minimum. Najbezpieczniej wobec tego byłoby wybrać możliwie najmniejszy parametr __beta__ i możliwie najwięcej iteracji, co oczywiście nie jest możliwe jeśli zależny nam na czasie. Problem dotyczący wpadania w minima lokalne można próbować rozwiązać stosując stochastyczny algorytm spadku gradientu, który zamiast używać dokładnie obliczonego gradientu, korzysta z aproksymowanego gradientu który posiada przez to losowe zakłócenia. Ze względu na to że algorytm nie porusza się wtedy dokładnie w kierunku spadku wartości funkcji optymalizowanej, istnieje szansa, że lokalne minima zostaną ominięte. 